# Notes
- simple article: https://www.geeksforgeeks.org/introduction-to-beam-search-algorithm/
- many beam search descriptions are given in terms of NLP context
- I think A* might not be feasible for large graphs, but I'm not sure...
- It'd be nice to code up the Clifford problem in a nice object-oriented with that made the graph structure abstracted, so I could use any old graph traversal algorithm

In [1]:
import tqdm
from typing import Dict
import numpy as np
import torch
import torch.nn as nn
import rubiks.clifford as cl
import rubiks.lgf as lgf
from qiskit.quantum_info import Clifford
import cProfile

In [2]:
num_qubits = 6

SEED = 123
use_qiskit = False
device = torch.device('cpu')
drop_phase_bits = True
scaling = 'log-linear'
data_dir = f"data/data_n_{num_qubits}_drop_phase_bits_scaling_{scaling}/"
high = cl.max_random_sequence_length(num_qubits, scaling)

lgf_model = lgf.LGFModel(
    num_qubits=num_qubits,
    device=device,
    rng=np.random.default_rng(SEED),
    hidden_layers=[32, 16, 4, 2],
    drop_phase_bits=drop_phase_bits,
    use_qiskit=use_qiskit,
).to(device)

lgf_model.load_state_dict(torch.load(data_dir + "checkpoint"))

<All keys matched successfully>

- Put this in the same format as other hillclimbing function
- Compare for different values of beam_width
    - should agree with previous results for beam_width=1
    - should be better for larger beam_widths
- Can it be made faster?
- Could add more tests
    - loading a Clifford from a bitstring
    - printing bitstring from Clifford
- Check that the final move_history is indeed a solution before exiting
- check hillclimbing/beamsearch for dropping phase bits and not dropping them

In [34]:
sequence_length = 15
initial_sequence = cl.random_sequence(np.random.default_rng(), seq_length=sequence_length, num_qubits=num_qubits)
initial_state = cl.sequence_to_tableau(initial_sequence, num_qubits)
results_dict = lgf.beam_search(initial_state=initial_state, lgf_model=lgf_model, beam_width=5, max_iter=1000, drop_phase_bits=True)

count = results_dict['count']
success = results_dict['success']

if success is False:
    print(f"No solution found after {count+1} steps")
else:
    print(f"Solution found after {count+1} steps")
    print(results_dict['move_history'])

Solution found after 5 steps
[(3, 5, 'swap'), (0, 4, 'swap'), (1, 'sdg'), (0, 1, 'cx'), (5, 'h')]


In [35]:
state = initial_state @ cl.sequence_to_tableau(results_dict['move_history'], num_qubits=num_qubits)
np.array_equal(state.tableau[:,:-1], cl.sequence_to_tableau([], num_qubits=num_qubits).tableau[:,:-1])

True

In [36]:
results_dict_hillclimbing = lgf.hillclimbing(initial_state=initial_state, lgf_model=lgf_model, max_iter=1000)
if results_dict_hillclimbing['success']:
    print(f"found solution in {len(results_dict_hillclimbing['move_history'])} steps")
    print(results_dict_hillclimbing['move_history'])

found solution in 5 steps
[(3, 5, 'swap'), (0, 4, 'swap'), (1, 'sdg'), (0, 1, 'cx'), (5, 'h')]


In [37]:
state = initial_state @ cl.sequence_to_tableau(results_dict_hillclimbing['move_history'], num_qubits=num_qubits)
np.array_equal(state.tableau[:,:-1], cl.sequence_to_tableau([], num_qubits=num_qubits).tableau[:,:-1])

True

In [ ]:
# get the move history to be in the same format
state = initial_state @ cl.sequence_to_tableau(results_dict_hillclimbing['move_history'][::-1], num_qubits=num_qubits)
state == cl.sequence_to_tableau([], num_qubits=num_qubits)

In [ ]:
cProfile.run('lgf.beam_search(initial_state=initial_state, lgf_model=lgf_model, beam_width=5, max_iter=1000)')